## Data Loading
Blah

In [3]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing

train = pd.DataFrame.from_csv('../../train.csv')
test = pd.DataFrame.from_csv('../../test.csv')

## Dimensions of train set
ntrain,dtrain = train.shape

## Dimensions of test set
ntest, dtest = test.shape

X = train.drop(['TARGET'], axis=1)

## Add a column that tells us how many 0's we have
X['n0'] = (X == 0).sum(axis=1)

Xtest = test
Xtest['n0'] = (Xtest == 0).sum(axis=1)

## Standardize sets together
def standardize_sets(X, Xtest):
    Xtotal = X.append(Xtest)
    Xtotal_scaled = preprocessing.scale(Xtotal)
    X_scaled,Xtest_scaled = np.split(Xtotal_scaled, [ntrain])
    return [X_scaled, Xtest_scaled]

targets = np.array(train.TARGET)

## Feature Selection

In [4]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif,chi2
from sklearn.preprocessing import Binarizer, scale

## Top 75th percentile
p = 75
y = targets

X_bin = Binarizer().fit_transform(scale(X))
selectChi2 = SelectPercentile(chi2, percentile=p).fit(X_bin, y)
selectF_classif = SelectPercentile(f_classif, percentile=p).fit(X, y)

chi2_selected = selectChi2.get_support()
chi2_selected_features = [ f for i,f in enumerate(X.columns) if chi2_selected[i]]
f_classif_selected = selectF_classif.get_support()
f_classif_selected_features = [ f for i,f in enumerate(X.columns) if f_classif_selected[i]]
selected = chi2_selected & f_classif_selected
features = [ f for f,s in zip(X.columns, selected) if s]

## Number of features selected
print(len(features))

X_selected = X[features]
Xtest_selected = Xtest[features]

X_selected,Xtest_selected = standardize_sets(X_selected, Xtest_selected)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python2.7/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 21  22  56  57  58  59  80  84  85 131 132 133 134 155 161 162 179 180
 189 192 220 222 234 238 244 248 261 262 303 307 315 319 327 349] are constant.
  UserWarning)


261


## Standardize data

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


## Make Validation Set

In [5]:
from sklearn import cross_validation

sss = cross_validation.StratifiedShuffleSplit(targets, 1, test_size=0.4, random_state=4224)
(train_index,valid_index) = list(sss)[0]

X_train = X_selected[train_index,]
X_valid = X_selected[valid_index,]
y_train = targets[train_index]
y_valid = targets[valid_index]

print(X_train.shape)
print(X_valid.shape)

(45612, 261)
(30408, 261)


## Run the thing (Linear)

In [6]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

lin_clf = svm.LinearSVC(dual=False, class_weight="balanced")
lin_clf.fit(X_train, y_train)

y_hat = lin_clf.predict(X_valid)
y_hat2 = lin_clf.predict(X_train)
print("Training accuracy:",lin_clf.score(X_train, y_train))
print("Validation accuracy:",lin_clf.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.68006226431640793)
('Validation accuracy:', 0.68284661931070767)
('Training AUC:', 0.71710942236112862)
('Validation AUC:', 0.7153391132680198)


In [12]:
preds = 1./(1.+np.exp(-y_hat))
y_hatp = np.vstack((1-preds, preds)).T
print("Validation AUC:",roc_auc_score(y_valid, y_hatp[:,1]))

('Validation AUC:', 0.7153391132680198)


## Run the thing (Polynomial kernel)
This takes almost forever to run

In [8]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="poly", degree=3, class_weight="balanced")
clfp.fit(X_train, y_train)

y_hat = clfp.predict(X_valid)
y_hat2 = clfp.predict(X_train)
print("Training accuracy:",clfp.score(X_train, y_train))
print("Validation accuracy:",clfp.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.75419297553275455)
('Validation accuracy:', 0.74026571954748754)
('Training AUC:', 0.72398845806217216)
('Validation AUC:', 0.66330910388191489)


## Run the thing (RBF kernel)
This also takes almost forever to run

In [10]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="rbf", class_weight="balanced")
clfp.fit(X_train, y_train)

y_hat = clfp.predict(X_valid)
y_hat2 = clfp.predict(X_train)
print("Training accuracy:",clfp.score(X_train, y_train))
print("Validation accuracy:",clfp.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.70057221783741119)
('Validation accuracy:', 0.69060773480662985)
('Training AUC:', 0.75684607177485153)
('Validation AUC:', 0.68205056331881908)
